In [1]:
# Import Splinter and BS
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)

In [4]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
# searching for elements with a specific combination of tag (ul and li) and attribute (item_list and slide)
# Also telling browser to wait one second before searching for components to ensure all parts are loaded
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [5]:
# Set up HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')

# Assign slide_elem as the variable to look for the ul tag and it's "descendent", item_list and the same for li and slide
# CSS works from right to left, returning the last item instead of the first so the first output will be the li.slide
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [7]:
# tell slide_elem to find the content title 
slide_elem.find("div", class_= 'content_title')

<div class="content_title"><a href="/news/8759/nasas-new-mars-rover-will-use-x-rays-to-hunt-fossils/" target="_self">NASA's New Mars Rover Will Use X-Rays to Hunt Fossils</a></div>

In [8]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's New Mars Rover Will Use X-Rays to Hunt Fossils"

In [9]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"PIXL, an instrument on the end of the Perseverance rover's arm, will search for chemical fingerprints left by ancient microbes."

### Featured Images

In [12]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [13]:
# find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [15]:
# Find the more info button and click
#  is element present by text method searches for an element that has the provided text, wait 1s again for loading
# This would return a boolean
browser.is_element_present_by_text('more info', wait_time=1)

# find by partial text methjod takes the string we found and locates the link associated with it 
more_info_elem = browser.links.find_by_partial_text('more info')

# tell splinter to click
more_info_elem.click()

In [18]:
# Now that the page is loaded, parse it 
html = browser.html
img_soup = soup(html, 'html.parser')

# find the relative image url
# figure.lede references the figure tag and it's class lede
# a is the next tag nested inside figure tag
# img tag is also nested here
# .get pulls the link to the image
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA18049_hires.jpg'

In [21]:
# add the base URL to create an absolute url
img_url = f'http://www.jpl.nasa.gov{img_url_rel}'
img_url

'http://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18049_hires.jpg'

### Mars Facts

In [14]:
# instead of scraping each row (td) in the table, use pd read_html and store in df
# read_html searches for and returns a list of tables. By specifying index 0, we are getting the first it encounters
df = pd.read_html('http://space-facts.com/mars/')[0]

# assign columns to the new df
df.columns=['description','value']
# assign index
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [26]:
# convert df back to html with pd.to_html()
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [27]:
# Close auto browser session
browser.quit()